In [1]:
import os, sys
sys.path.append(os.path.abspath('../../../main/python'))

In [2]:
import thalesians.tsa.conversions as conv
import thalesians.tsa.datasets.finratrace as finratrace

import importlib
importlib.reload(finratrace)

<module 'thalesians.tsa.datasets.finratrace' from 'S:\\dev\\tsa\\src\\main\\python\\thalesians\\tsa\\datasets\\finratrace.py'>

In [8]:
data_file_path = 'S:/Dropbox/fifteen-us-companies-trace-bond-trades.zip'
first_report_date = conv.str_to_date('2015.01.01')
last_report_date = conv.str_to_date('2015.12.31')
print(first_report_date, last_report_date)

2015-01-01 2015-12-31


In [7]:
df = finratrace.load_df_from_file(data_file_path, cusip='060505DP6', first_report_date=first_report_date, last_report_date=last_report_date)

In [17]:
len(df)

3234

In [22]:
import importlib
import thalesians.tsa.pandasutils as pdutils
importlib.reload(pdutils)
pdutils.detect_df_column_types(df)

cusip_id
bond_sym_id
company_symbol
bsym
trd_exctn_dt
trd_exctn_tm
sub_prd_type
trans_dt
msg_seq_nb
trc_st
wis_fl
cmsn_trd
remuneration
ascii_rptd_vol_tx
frmt_cd
rptd_pr
yld_sign_cd
yld_pt
asof_cd
days_to_sttl_ct
sale_cndtn_cd
sale_cndtn2_cd
spcl_trd_fl
diss_rptg_side_cd
side
chng_cd
rptd_high_pr
high_yld_sign_cd
high_yld_pt
rptd_low_pr
low_yld_sign_cd
low_yld_pt
rptd_last_pr
lsal_yld_sign_cd
lsal_yld_pt
orig_dis_dt
orig_msg_seq_nb
function
sttl_dt
rptg_party_type
contra_party_type
ATS_indicator


In [15]:
df.columns

Index(['cusip_id', 'bond_sym_id', 'company_symbol', 'bsym', 'trd_exctn_dt',
       'trd_exctn_tm', 'sub_prd_type', 'trans_dt', 'msg_seq_nb', 'trc_st',
       'wis_fl', 'cmsn_trd', 'remuneration', 'ascii_rptd_vol_tx', 'frmt_cd',
       'rptd_pr', 'yld_sign_cd', 'yld_pt', 'asof_cd', 'days_to_sttl_ct',
       'sale_cndtn_cd', 'sale_cndtn2_cd', 'spcl_trd_fl', 'diss_rptg_side_cd',
       'side', 'chng_cd', 'rptd_high_pr', 'high_yld_sign_cd', 'high_yld_pt',
       'rptd_low_pr', 'low_yld_sign_cd', 'low_yld_pt', 'rptd_last_pr',
       'lsal_yld_sign_cd', 'lsal_yld_pt', 'orig_dis_dt', 'orig_msg_seq_nb',
       'function', 'sttl_dt', 'rptg_party_type', 'contra_party_type',
       'ATS_indicator'],
      dtype='object')

In [13]:
df.head()

,cusip_id,bond_sym_id,company_symbol,bsym,trd_exctn_dt,trd_exctn_tm,sub_prd_type,trans_dt,msg_seq_nb,trc_st,...,rptd_last_pr,lsal_yld_sign_cd,lsal_yld_pt,orig_dis_dt,orig_msg_seq_nb,function,sttl_dt,rptg_party_type,contra_party_type,ATS_indicator
0,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/01/02,12:10:56,CORP,2015/01/02,8057,G,...,,,,,,,2015/01/07,,,
1,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/01/02,13:16:08,CORP,2015/01/02,10473,G,...,,,,,,,2015/01/07,,,
2,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/01/02,14:37:22,CORP,2015/01/02,13270,G,...,,,,,,,2015/01/07,,,
3,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/01/02,14:37:22,CORP,2015/01/02,13271,G,...,,,,,,,2015/01/07,,,
4,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/01/02,15:01:02,CORP,2015/01/02,14072,G,...,,,,,,,2015/01/07,,,


In [14]:
df.tail()

,cusip_id,bond_sym_id,company_symbol,bsym,trd_exctn_dt,trd_exctn_tm,sub_prd_type,trans_dt,msg_seq_nb,trc_st,...,rptd_last_pr,lsal_yld_sign_cd,lsal_yld_pt,orig_dis_dt,orig_msg_seq_nb,function,sttl_dt,rptg_party_type,contra_party_type,ATS_indicator
3229,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/12/29,13:11:50,CORP,2015/12/29,21305,M,...,,,,,,,2016/01/04,D,C,
3230,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/12/30,10:11:19,CORP,2015/12/30,4491,M,...,,,,,,,2016/01/05,D,A,
3231,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/12/31,8:17:02,CORP,2015/12/31,191,M,...,,,,,,,2016/01/06,D,D,
3232,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/12/31,8:17:38,CORP,2015/12/31,189,M,...,,,,,,,2016/01/06,D,C,
3233,060505DP6,BAC.HBM,BAC,BBG0000LFX94,2015/12/31,13:02:50,CORP,2015/12/31,12666,M,...,,,,,,,2016/01/06,D,C,
